## ServiceTitan Internship Round 1 Task

In [6]:
import pickle
import pandas as pd
from datetime import datetime

**As seen below, the original data has several inconsistencies:**

* There are Invoice IDs in the data that have letters at the end.
* There are examples in 'quantity' that have word numbers instead of integers in the values, e.g. 'ten' instead of 10.
* In 'type' there is a letter instead of numbers.
* There are two out-of-range date examples which show February 30th.

In [2]:
with open("data\invoices_new.pkl", 'rb') as f:
    data = pickle.load(f)

In [3]:
data

[{'id': '365371O',
  'created_on': '2019-05-27',
  'items': [{'item': {'id': 142446,
     'name': 'ii_142446',
     'unit_price': 182,
     'type': 1},
    'quantity': 6},
   {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1},
    'quantity': 1}]},
 {'id': 389528,
  'created_on': '2019-03-19',
  'items': [{'item': {'id': 172535,
     'name': 'ii_172535',
     'unit_price': 105,
     'type': 0},
    'quantity': 4},
   {'item': {'id': 175208, 'name': 'ii_175208', 'unit_price': 109, 'type': 0},
    'quantity': 7},
   {'item': {'id': 139970, 'name': 'ii_139970', 'unit_price': 193, 'type': 2},
    'quantity': 4},
   {'item': {'id': 156273, 'name': 'ii_156273', 'unit_price': 129, 'type': 2},
    'quantity': 6},
   {'item': {'id': 178512, 'name': 'ii_178512', 'unit_price': 108, 'type': 1},
    'quantity': 7},
   {'item': {'id': 191045, 'name': 'ii_191045', 'unit_price': 195, 'type': 1},
    'quantity': 8},
   {'item': {'id': 189841, 'name': 'ii_189841', 'unit_price': 1

**The DataExtractor class below implements functionalities that handle all the inconsistencies in the data, clean and process it, and transform it into a pandas Data Frame in the correct format.**

In [35]:
class DataExtractor:
    """
    Loads the existing data and transforms into the correct format.
    """
    
    def __init__(self):
        self.invoices, self.expired_invoices = self.load_data()
        self.data = self.transform_data(self.invoices, self.expired_invoices)
    
    def load_data(self):
        """
        Loads the new invoice data from the .pkl file and the expired invoide data from the .txt file.
        """

        with open("data\invoices_new.pkl", 'rb') as f:
            invoices = pickle.load(f)

        with open("data\expired_invoices.txt", 'r') as f:
            expired_invoices = set(map(int, f.read().split(', ')))
            
        return invoices, expired_invoices
    
    def clean_invoice_id(self, invoice_id):
        """
        Removes letters at the end of the invoice IDs.
        """
        return int(''.join(filter(str.isdigit, str(invoice_id))))
    
    def convert_quantity(self, quantity):
        """
        Converts the quantity to integers.
        """
        if isinstance(quantity, int):
            return quantity
        if quantity.isdigit():
            return int(quantity)
        
        quantity = quantity.lower()
        word_numbers = {
            'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4,
            'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9,
            'ten': 10} # example for up to ten numbers
        
        return word_numbers.get(quantity, 0)  # default to 0 if not found
    
    def convert_type(self, type_value):
        """
        Converts the type to a proper integer.
        """
        if type_value == 'O':
            return 0
        return int(type_value)
    
    def correct_date(self, date_str):
        """
        Handles the out-of-range date to the nearest valid date.
        """
        try:
            return pd.to_datetime(date_str)
        except ValueError:
            date_parts = date_str.split('-')
            year, month, day = map(int, date_parts)
            if month == 2 and day > 28:
                # Leap year
                if year % 4 == 0: #and (year % 100 != 0 or year % 400 == 0):
                    day = 29 
                else:
                    day = 28  # Non-leap year
            return pd.Timestamp(year=year, month=month, day=day)
    
    def transform_data(self, invoices, expired_invoices):
        """
        Transforms the data into the final format, creating a pandas Data Frame.
        """
        
        transformed_data = []
        
        type_conversion = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'} # to convert the type column
        
        for invoice in invoices:
            invoice_id = self.clean_invoice_id(invoice['id'])
            created_on = self.correct_date(invoice['created_on'])
            
            if 'items' not in invoice or not invoice['items']:
                continue # ensure that there is 'items' before proceeding
            invoice_total = sum(item['item']['unit_price'] * self.convert_quantity(item['quantity']) for item in invoice['items']) # later used in finding percentage
            
            is_expired = invoice_id in expired_invoices
        
            for item in invoice['items']:
                invoiceitem_id = int(item['item']['id'])
                invoiceitem_name = str(item['item']['name'])
                item_type_value = item['item']['type']
                item_type = str(type_conversion[self.convert_type(item_type_value)])
                unit_price = int(item['item']['unit_price'])
                quantity = self.convert_quantity(item['quantity'])
                total_price = unit_price * quantity
                percentage_in_invoice = float(total_price / invoice_total)

                transformed_data.append({
                    'invoice_id': invoice_id,
                    'created_on': created_on,
                    'invoiceitem_id': invoiceitem_id,
                    'invoiceitem_name': invoiceitem_name,
                    'type': item_type,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'percentage_in_invoice': percentage_in_invoice,
                    'is_expired': is_expired
                })
            
        data = pd.DataFrame(transformed_data)
        
        return data

In [36]:
extractor = DataExtractor()
df = extractor.data

In [37]:
df

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,percentage_in_invoice,is_expired
0,365371,2019-05-27,142446,ii_142446,Equipment,182,1092,0.861199,False
1,365371,2019-05-27,154019,ii_154019,Equipment,176,176,0.138801,False
2,389528,2019-03-19,172535,ii_172535,Material,105,420,0.073311,True
3,389528,2019-03-19,175208,ii_175208,Material,109,763,0.133182,True
4,389528,2019-03-19,139970,ii_139970,Service,193,772,0.134753,True
...,...,...,...,...,...,...,...,...,...
487,321243,2019-05-26,175528,ii_175528,Equipment,169,507,0.232037,False
488,377960,2019-02-22,123242,ii_123242,Equipment,110,550,0.164720,True
489,377960,2019-02-22,196386,ii_196386,Other,150,900,0.269542,True
490,377960,2019-02-22,196707,ii_196707,Service,191,573,0.171608,True


In [38]:
df.dtypes

invoice_id                        int64
created_on               datetime64[ns]
invoiceitem_id                    int64
invoiceitem_name                 object
type                             object
unit_price                        int64
total_price                       int64
percentage_in_invoice           float64
is_expired                         bool
dtype: object

In [39]:
df.to_csv("DataFrame.csv")